# Installs

In [35]:
# !pip install Pillow
# !pip install torchvision --index-url https://download.pytorch.org/whl/cu118
# !pip install -r requirements.txt
# !pip install pandas==2.3.2
# !pip install scikit-learn==1.7.1
# !pip install matplotlib==3.10.5
# !pip install shap
# !pip install lime
# !pip install torchcam 
# !pip install numpy
# !pip install numpy==1.25.2
# !pip install torchcam
# !pip install torchcam -f https://download.pytorch.org/whl/torch_stable.html

  Using cached torchcam-0.4.1-py3-none-any.whl.metadata (30 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + C:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Scripts\python.exe C:\Users\nalex\AppData\Local\Temp\pip-install-fzulp1g4\numpy_e67756dcafb04982b844b724a627ecdd\vendored-meson\meson\meson.py setup C:\Users\nalex\AppData\Local\Temp\pip-install-fzulp1g4\numpy_e67756dcafb04982b844b724a627ecdd C:\Users\nalex\AppData\Local\Temp\pip-install-fzulp1g4\numpy_e67756dcafb04982b844b724a627ecdd\.mesonpy-z7wnn259 -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\nalex\AppData\Local\Temp\pip-install-fzulp1g4\numpy_e67756dcafb04982b844b724a627ecdd\.mesonpy-z7wnn259\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\nalex\AppData\Local\Temp\pip-install-fzulp1g4\numpy_e67756dcafb04982b844b724a627ecdd
      Build dir: C:\Users\nalex\AppData\Lo

# Augmentation and Folds

In [ ]:
# Script to rename images in each 'original' subfolder by category
import os
from pathlib import Path

# Datasets
roots = [
    Path("../data/imgShiny/original"),
    Path("../data/png4ch/original")
 ]

for root in roots:
    print(f"Processing: {root}")
    # Iterate over each category subfolder
    for category in os.listdir(root):
        cat_dir = root / category
        if not cat_dir.is_dir():
            continue
        # List all image files in the category folder
        images = sorted([p for p in cat_dir.iterdir() if p.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']])
        for idx, img_path in enumerate(images, 1):
            # Build new filename: category_001.ext, category_002.ext, ...
            new_name = f"{category}_{idx:03d}{img_path.suffix.lower()}"
            new_path = cat_dir / new_name
            # Rename if the filename is different
            if img_path != new_path:
                os.rename(img_path, new_path)
        print(f"  {category}: {len(images)} images renamed.")
print("Renaming completed.")

Procesando: ..\data\imgShiny\original
  absence: 80 imágenes renombradas.
  presence-defect: 203 imágenes renombradas.
  presence-good: 51 imágenes renombradas.
Procesando: ..\data\png4ch\original
  absence: 80 imágenes renombradas.
  presence-defect: 206 imágenes renombradas.
  presence-good: 51 imágenes renombradas.
Renombrado completado.


In [ ]:
def augment_images(original_root, augmented_root, target_count):
    """
    Performs image augmentation in the folders of original_root and saves to augmented_root.
    - original_root: input path with subfolders per class.
    - augmented_root: output path where augmented images will be saved.
    - target_count: total number of images per class after augmentation.
    """
    import os
    from pathlib import Path
    from PIL import Image
    import random
    from torchvision import transforms

    original_root = Path(original_root)
    augmented_root = Path(augmented_root)

    # Find categories
    categories = [d for d in os.listdir(original_root) if (original_root / d).is_dir()]

    for cat in categories:
        orig_dir = original_root / cat
        aug_dir = augmented_root / cat
        aug_dir.mkdir(parents=True, exist_ok=True)

        # List all valid image files
        images = [p for p in orig_dir.glob('*') if p.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']]
        n_orig = len(images)

        # Detect mode of the first valid image (RGB or RGBA)
        mode = None
        for img_path in images:
            try:
                img = Image.open(img_path)
                if img.mode in ['RGB', 'RGBA']:
                    mode = img.mode
                    break
                else:
                    print(f"[WARNING] Image ignored due to unsupported mode: {img_path} (mode: {img.mode})")
            except Exception as e:
                print(f"[ERROR] Could not open {img_path}: {e}")

        if mode is None:
            print(f"[WARNING] No RGB or RGBA images found in category {cat}. Skipping.")
            continue

        # Define augmentation pipeline according to image mode
        if mode == 'RGBA':
            augment = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(degrees=15),
                transforms.RandomResizedCrop(size=224, scale=(0.9, 1.0)),
            ])
        else:
            augment = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(degrees=15),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
                transforms.RandomResizedCrop(size=224, scale=(0.9, 1.0)),
            ])

        # Copy valid originals
        valid_images = []
        for idx, img_path in enumerate(images):
            try:
                img = Image.open(img_path)
                if img.mode == mode:
                    img.save(aug_dir / f'orig_{idx+1:04d}.png')
                    valid_images.append(img_path)
                else:
                    print(f"[WARNING] Image ignored due to different mode: {img_path} (mode: {img.mode})")
            except Exception as e:
                print(f"[ERROR] Could not open {img_path}: {e}")

        n_valid = len(valid_images)

        # Augment until target_count
        extra_needed = target_count - n_valid
        if extra_needed > 0 and valid_images:
            for i in range(extra_needed):
                src_path = random.choice(valid_images)
                img = Image.open(src_path).convert(mode)
                aug_img = augment(img)
                aug_img.save(aug_dir / f'aug_{i+1:04d}.png')

        print(f'Category: {cat} | Valid originals: {n_valid} | Total saved: {max(target_count, n_valid)} in {aug_dir}')

In [ ]:
import os
import shutil
from pathlib import Path
import random

# Params
# original_root = Path("../data/imgShiny/original")
# output_root = Path("../data/imgShiny/224px")
original_root = Path("../data/png4ch/original")
output_root = Path("../data/png4ch/224px")
n_folds = 5
split_ratio = [0.7, 0.15, 0.15]  # train, val, test

# Get categories
categories = [d for d in os.listdir(original_root) if (original_root / d).is_dir()]

for fold in range(1, n_folds+1):
    fold_dir = output_root / f"orFold{fold}"
    for split, ratio in zip(['train', 'val', 'test'], split_ratio):
        for cat in categories:
            split_dir = fold_dir / split / cat
            split_dir.mkdir(parents=True, exist_ok=True)

    # For each class, shuffle and split images
    for cat in categories:
        img_dir = original_root / cat
        images = [p for p in img_dir.glob('*') if p.suffix.lower() in ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']]
        random.shuffle(images)
        n_total = len(images)
        n_train = int(n_total * split_ratio[0])
        n_val = int(n_total * split_ratio[1])
        n_test = n_total - n_train - n_val

        splits = {
            'train': images[:n_train],
            'val': images[n_train:n_train+n_val],
            'test': images[n_train+n_val:]
        }

        for split, img_list in splits.items():
            for img_path in img_list:
                dest = output_root / f"orFold{fold}" / split / cat / img_path.name
                shutil.copy2(img_path, dest)

    print(f"Fold {fold} created in {fold_dir}")

Fold 1 creado en ..\data\png4ch\224px\orFold1
Fold 2 creado en ..\data\png4ch\224px\orFold2
Fold 2 creado en ..\data\png4ch\224px\orFold2
Fold 3 creado en ..\data\png4ch\224px\orFold3
Fold 3 creado en ..\data\png4ch\224px\orFold3
Fold 4 creado en ..\data\png4ch\224px\orFold4
Fold 4 creado en ..\data\png4ch\224px\orFold4
Fold 5 creado en ..\data\png4ch\224px\orFold5
Fold 5 creado en ..\data\png4ch\224px\orFold5


In [ ]:
# Use augment_images to create train_aug in each fold
import os
from pathlib import Path

# Base path for folds
# base_folds = Path("../data/imgShiny/224px")
base_folds = Path("../data/png4ch/224px")
n_folds = 5
target_count = 1000 

for fold in range(1, n_folds+1):
    fold_dir = base_folds / f"orFold{fold}"
    train_dir = fold_dir / "train"
    train_aug_dir = fold_dir / "train_aug"
    print(f"Augmenting images in: {train_dir} -> {train_aug_dir}")
    augment_images(str(train_dir), str(train_aug_dir), target_count)

Aumentando imágenes en: ..\data\png4ch\224px\orFold1\train -> ..\data\png4ch\224px\orFold1\train_aug
Categoría: absence | Originales válidas: 56 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\absence
Categoría: absence | Originales válidas: 56 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\absence
Categoría: presence-defect | Originales válidas: 144 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\presence-defect
Categoría: presence-defect | Originales válidas: 144 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\presence-defect
Categoría: presence-good | Originales válidas: 35 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\presence-good
Aumentando imágenes en: ..\data\png4ch\224px\orFold2\train -> ..\data\png4ch\224px\orFold2\train_aug
Categoría: presence-good | Originales válidas: 35 | Total guardadas: 1000 en ..\data\png4ch\224px\orFold1\train_aug\presence-good
Aumentando imágenes en: ..\data\

In [3]:
!pip install torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 536.2/536.2 kB 13.2 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 5.5/5.5 MB 34.6 MB/s  0:00:00
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ----------------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Model Training

In [ ]:
# ==============================
# Grid Search con pesos preentrenados (excepto SimpleCNN)
# ==============================
import os
import glob
import time
import datetime
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import vit_b_16

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configuración general
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

torch.backends.cudnn.benchmark = True

# root_fold = "../data/imgShiny/224px/orFold1"
foldName = "orFold2"
root_fold = f"../data/imgShiny/224px/{foldName}"
train_dir = os.path.join(root_fold, "train_aug")
val_dir   = os.path.join(root_fold, "val")
test_dir  = os.path.join(root_fold, "test")

classes = ['presence-defect', 'presence-good', 'absence']
num_classes = len(classes)

os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

timestamp_global = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_out_pretrained = f"../results/metrics_pretrained_{timestamp_global}_{foldName}.csv"
csv_out_curves = f"../results/curves_pretrained_{timestamp_global}_{foldName}.csv"

def make_transforms(image_size):
    train_tfms = transforms.Compose([
        transforms.RandomResizedCrop(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    val_test_tfms = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return train_tfms, val_test_tfms

# Red neuronal simple para imágenes
class SimpleCNN(nn.Module):
    def __init__(self, num_classes, image_size):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (image_size // 8) * (image_size // 8), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# build_model con pesos preentrenados excepto simplecnn
def build_model_pretrained(arch_name: str, num_classes: int, image_size=224, pretrained_models_cache=None):
    if pretrained_models_cache is not None and arch_name in pretrained_models_cache:
        # Usar modelo cacheado (sin la última capa)
        net = pretrained_models_cache[arch_name]
        # Reemplazar la última capa según arquitectura
        if arch_name == 'mobilenet':
            in_features = net.classifier[1].in_features
            net.classifier[1] = nn.Linear(in_features, num_classes)
        elif arch_name == 'vgg16':
            in_features = net.classifier[6].in_features
            net.classifier[6] = nn.Linear(in_features, num_classes)
        elif arch_name == 'resnet50':
            in_features = net.fc.in_features
            net.fc = nn.Linear(in_features, num_classes)
        elif arch_name == 'vit':
            in_features = net.heads.head.in_features
            net.heads.head = nn.Linear(in_features, num_classes)
        return net
    # Si no está en cache, cargar y cachear
    if arch_name == 'mobilenet':
        net = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        if pretrained_models_cache is not None:
            pretrained_models_cache[arch_name] = net
        in_features = net.classifier[1].in_features
        net.classifier[1] = nn.Linear(in_features, num_classes)
    elif arch_name == 'vgg16':
        net = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        if pretrained_models_cache is not None:
            pretrained_models_cache[arch_name] = net
        in_features = net.classifier[6].in_features
        net.classifier[6] = nn.Linear(in_features, num_classes)
    elif arch_name == 'resnet50':
        net = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        if pretrained_models_cache is not None:
            pretrained_models_cache[arch_name] = net
        in_features = net.fc.in_features
        net.fc = nn.Linear(in_features, num_classes)
    elif arch_name == 'vit':
        net = vit_b_16(weights="IMAGENET1K_V1")
        if pretrained_models_cache is not None:
            pretrained_models_cache[arch_name] = net
        in_features = net.heads.head.in_features
        net.heads.head = nn.Linear(in_features, num_classes)
    elif arch_name == 'simplecnn':
        net = SimpleCNN(num_classes, image_size)
    else:
        raise ValueError(f"Arquitectura no soportada: {arch_name}")
    return net

def evaluate_on_loader(model, loader, device=device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy().tolist())
            y_pred.extend(preds.cpu().numpy().tolist())

    if len(y_true) != len(y_pred):
        print(f"[WARN] Longitudes distintas en evaluación: y_true={len(y_true)} vs y_pred={len(y_pred)}. Ajustando a longitud mínima.")
        m = min(len(y_true), len(y_pred))
        y_true = y_true[:m]
        y_pred = y_pred[:m]

    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec  = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1   = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    return acc, prec, rec, f1

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=200, patience=10, device=device):
    best_f1 = -1.0
    best_state = None
    patience_counter = 0
    best_epoch = 0

    # Para guardar curvas
    history = {
        'epoch': [],
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'train_prec': [],
        'val_prec': [],
        'train_rec': [],
        'val_rec': [],
        'train_f1': [],
        'val_f1': []
    }

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        train_preds, train_labels = [], []

        for inputs, labels in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            train_preds.extend(preds.cpu().numpy().tolist())
            train_labels.extend(labels.cpu().numpy().tolist())

        # Métricas de train
        if len(train_labels) != len(train_preds):
            m = min(len(train_labels), len(train_preds))
            train_labels = train_labels[:m]
            train_preds = train_preds[:m]
        train_acc  = accuracy_score(train_labels, train_preds)
        train_prec = precision_score(train_labels, train_preds, average='weighted', zero_division=0)
        train_rec  = recall_score(train_labels, train_preds, average='weighted', zero_division=0)
        train_f1   = f1_score(train_labels, train_preds, average='weighted', zero_division=0)

        model.eval()
        val_running_loss = 0.0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy().tolist())
                val_labels.extend(labels.cpu().numpy().tolist())

        if len(val_labels) != len(val_preds):
            print(f"[WARN] Longitudes distintas en validación: y_true={len(val_labels)} vs y_pred={len(val_preds)}. Ajustando a longitud mínima.")
            m = min(len(val_labels), len(val_preds))
            val_labels = val_labels[:m]
            val_preds  = val_preds[:m]

        val_acc  = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_rec  = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_f1   = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

        # Guardar en historia
        history['epoch'].append(epoch)
        history['train_loss'].append(running_loss)
        history['val_loss'].append(val_running_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_prec'].append(train_prec)
        history['val_prec'].append(val_prec)
        history['train_rec'].append(train_rec)
        history['val_rec'].append(val_rec)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch:02d} | train_loss: {running_loss:.4f} | val_loss: {val_running_loss:.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc:.4f} | val_F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
            best_epoch = epoch
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping activado.")
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    model.to(device)

    # Extraer métricas del mejor epoch
    idx = best_epoch - 1 if best_epoch > 0 else -1
    best_metrics = {
        'train_acc': history['train_acc'][idx],
        'val_acc': history['val_acc'][idx],
        'train_prec': history['train_prec'][idx],
        'val_prec': history['val_prec'][idx],
        'train_rec': history['train_rec'][idx],
        'val_rec': history['val_rec'][idx],
        'train_f1': history['train_f1'][idx],
        'val_f1': history['val_f1'][idx],
        'train_loss': history['train_loss'][idx],
        'val_loss': history['val_loss'][idx]
    }

    return model, best_f1, best_epoch, history, best_metrics

image_sizes   = [224]
batch_sizes   = [16, 32]
learning_rates= [1e-3, 1e-4]
# learning_rates= [5e-3]
epochs        = 200
patience      = 10

architectures = ['resnet50', 'mobilenet', 'simplecnn']
# architectures = ['vit', 'simplecnn']
# architectures = ['resnet50', 'mobilenet', 'vgg16', 'vit', 'simplecnn']
# architectures = ['simplecnn']

all_results = []
all_curves = []
fold_name = os.path.basename(root_fold.rstrip('/\\'))

# --- Cache de modelos preentrenados para evitar descargas repetidas ---
pretrained_models_cache = {}

for arch in architectures:
    for img_size in image_sizes:
        train_tfms, val_test_tfms = make_transforms(img_size)

        train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
        val_ds   = datasets.ImageFolder(val_dir,   transform=val_test_tfms)
        test_ds  = datasets.ImageFolder(test_dir,  transform=val_test_tfms)
        # model = build_model_pretrained(arch, num_classes, image_size=img_size).to(device)

        for bs in batch_sizes:
            train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True,  num_workers=0, pin_memory=True)
            val_loader   = DataLoader(val_ds,   batch_size=bs, shuffle=False, num_workers=0, pin_memory=True)
            test_loader  = DataLoader(test_ds,  batch_size=bs, shuffle=False, num_workers=0, pin_memory=True)

            for lr in learning_rates:
                print(f"\n=== Entrenando {arch} (pretrained) | img={img_size} | bs={bs} | lr={lr} ===")
                start_t = time.time()

                model = build_model_pretrained(arch, num_classes, image_size=img_size, pretrained_models_cache=pretrained_models_cache).to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                model, best_val_f1, best_epoch, history, best_metrics = train_model(
                    model,
                    nn.CrossEntropyLoss(),
                    optimizer,
                    train_loader, val_loader,
                    epochs=epochs, patience=patience, device=device
                )
                train_time = time.time() - start_t

                acc, prec, rec, f1 = evaluate_on_loader(model, test_loader, device=device)

                ts_conf = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

                model_path = f"../models/{foldName}/{arch}_pretrained_{ts_conf}_img{img_size}_bs{bs}_lr{lr}.pth"
                torch.save(model.state_dict(), model_path)

                row = {
                    "Fold": fold_name,
                    "Model": arch,
                    "ImageSize": img_size,
                    "BatchSize": bs,
                    "LR": lr,
                    "EpochsUsed": best_epoch,
                    "Patience": patience,
                    "ValF1Best": best_val_f1,
                    "TrainAccuracy": best_metrics['train_acc'],
                    "ValAccuracy": best_metrics['val_acc'],
                    "TrainPrecision": best_metrics['train_prec'],
                    "ValPrecision": best_metrics['val_prec'],
                    "TrainRecall": best_metrics['train_rec'],
                    "ValRecall": best_metrics['val_rec'],
                    "TrainF1": best_metrics['train_f1'],
                    "ValF1": best_metrics['val_f1'],
                    "TrainLoss": best_metrics['train_loss'],
                    "ValLoss": best_metrics['val_loss'],
                    "TestAccuracy": acc,
                    "TestPrecision": prec,
                    "TestRecall": rec,
                    "TestF1": f1,
                    "TrainTimeSec": round(train_time, 2),
                    "Timestamp": ts_conf,
                    "ModelPath": model_path
                }
                all_results.append(row)

                # Guardar curvas para graficar
                for i in range(len(history['epoch'])):
                    curve_row = {
                        "Fold": fold_name,
                        "Model": arch,
                        "ImageSize": img_size,
                        "BatchSize": bs,
                        "LR": lr,
                        "Epoch": history['epoch'][i],
                        "TrainLoss": history['train_loss'][i],
                        "ValLoss": history['val_loss'][i],
                        "TrainAccuracy": history['train_acc'][i],
                        "ValAccuracy": history['val_acc'][i],
                        "TrainPrecision": history['train_prec'][i],
                        "ValPrecision": history['val_prec'][i],
                        "TrainRecall": history['train_rec'][i],
                        "ValRecall": history['val_rec'][i],
                        "TrainF1": history['train_f1'][i],
                        "ValF1": history['val_f1'][i],
                        "Timestamp": ts_conf,
                        "ModelPath": model_path
                    }
                    all_curves.append(curve_row)

                df = pd.DataFrame(all_results)
                df.to_csv(csv_out_pretrained, index=False)

                df_curves = pd.DataFrame(all_curves)
                df_curves.to_csv(csv_out_curves, index=False)

                print(f"-> Test F1: {f1:.4f} | Val F1 (best): {best_val_f1:.4f} | modelo guardado: {model_path}")

best_by_arch = df.sort_values(by="ValF1Best", ascending=False).groupby("Model").head(1)
print("\nMejores configuraciones por arquitectura (pretrained, según ValF1Best):")
print(best_by_arch[["Fold","Model","ImageSize","BatchSize","LR","ValF1Best","TestF1","Timestamp","ModelPath"]])

Usando dispositivo: cuda

=== Entrenando resnet50 (pretrained) | img=224 | bs=16 | lr=0.001 ===
Epoch 01 | train_loss: 114.3150 | val_loss: 0.6813 | train_acc: 0.7353 | val_acc: 0.9388 | val_F1: 0.9359
Epoch 01 | train_loss: 114.3150 | val_loss: 0.6813 | train_acc: 0.7353 | val_acc: 0.9388 | val_F1: 0.9359
Epoch 02 | train_loss: 74.9507 | val_loss: 3.1267 | train_acc: 0.8407 | val_acc: 0.8571 | val_F1: 0.8439
Epoch 02 | train_loss: 74.9507 | val_loss: 3.1267 | train_acc: 0.8407 | val_acc: 0.8571 | val_F1: 0.8439
Epoch 03 | train_loss: 63.2328 | val_loss: 0.2494 | train_acc: 0.8627 | val_acc: 0.9592 | val_F1: 0.9580
Epoch 03 | train_loss: 63.2328 | val_loss: 0.2494 | train_acc: 0.8627 | val_acc: 0.9592 | val_F1: 0.9580
Epoch 04 | train_loss: 54.4170 | val_loss: 1.1131 | train_acc: 0.8853 | val_acc: 0.8163 | val_F1: 0.8240
Epoch 04 | train_loss: 54.4170 | val_loss: 1.1131 | train_acc: 0.8853 | val_acc: 0.8163 | val_F1: 0.8240
Epoch 05 | train_loss: 60.1600 | val_loss: 0.1984 | train_acc:

In [ ]:
# ==============================
# Grid Search para png4ch con proyección 4→3 canales (1x1 conv)
# ==============================
import os
import glob
import time
import datetime
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import vit_b_16

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

torch.backends.cudnn.benchmark = True

foldName = "orFold2"
root_fold = f"../data/png4ch/224px/{foldName}"
train_dir = os.path.join(root_fold, "train_aug")
val_dir   = os.path.join(root_fold, "val")
test_dir  = os.path.join(root_fold, "test")

classes = ['absence', 'presence-defect', 'presence-good']
num_classes = len(classes)

os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

timestamp_global = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_out_pretrained = f"../results/metrics_pretrained_png4ch_{timestamp_global}_{foldName}.csv"
csv_out_curves = f"../results/curves_pretrained_png4ch_{timestamp_global}_{foldName}.csv"

def pil_loader_rgba(path):
    img = Image.open(path)
    arr = np.array(img)
    if arr.ndim == 2:
        arr = np.stack([arr]*4, axis=-1)
    if arr.shape[2] == 3:
        # Si ya es RGB, agregar canal extra (dummy)
        arr = np.concatenate([arr, np.zeros_like(arr[..., :1])], axis=-1)
    elif arr.shape[2] == 1:
        arr = np.concatenate([arr]*4, axis=-1)
    elif arr.shape[2] != 4:
        raise ValueError(f"Imagen con {arr.shape[2]} canales no soportada")
    arr = arr.astype(np.float32) / 255.0
    tensor = torch.from_numpy(arr).permute(2,0,1)
    return tensor

class ImageFolderRGBA(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super().__init__(root, transform=transform)
        self.loader = pil_loader_rgba

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

def make_transforms(image_size):
    def train_tfms(x):
        x = transforms.RandomResizedCrop(image_size)(x)
        x = transforms.RandomHorizontalFlip()(x)
        x = transforms.RandomRotation(20)(x)
        return x
    def val_test_tfms(x):
        x = transforms.Resize((image_size, image_size))(x)
        return x
    return train_tfms, val_test_tfms

class FourToThreeConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Conv2d(4, 3, kernel_size=1, bias=True)
        with torch.no_grad():
            self.proj.weight[:] = 0.25
            self.proj.bias[:] = 0.0
    def forward(self, x):
        return self.proj(x)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes, image_size, in_channels=4):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (image_size // 8) * (image_size // 8), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def build_model_pretrained(arch_name: str, num_classes: int, image_size=224, pretrained_models_cache=None):
    if arch_name in ['mobilenet', 'vgg16', 'resnet50', 'vit']:
        proj = FourToThreeConv()
        if pretrained_models_cache is not None and arch_name in pretrained_models_cache:
            backbone = pretrained_models_cache[arch_name]
        else:
            if arch_name == 'mobilenet':
                backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
                in_features = backbone.classifier[1].in_features
                backbone.classifier[1] = nn.Linear(in_features, num_classes)
            elif arch_name == 'vgg16':
                backbone = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
                in_features = backbone.classifier[6].in_features
                backbone.classifier[6] = nn.Linear(in_features, num_classes)
            elif arch_name == 'resnet50':
                backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
                in_features = backbone.fc.in_features
                backbone.fc = nn.Linear(in_features, num_classes)
            elif arch_name == 'vit':
                backbone = vit_b_16(weights="IMAGENET1K_V1")
                in_features = backbone.heads.head.in_features
                backbone.heads.head = nn.Linear(in_features, num_classes)
            if pretrained_models_cache is not None:
                pretrained_models_cache[arch_name] = backbone
        net = nn.Sequential(proj, backbone)
    elif arch_name == 'simplecnn':
        net = SimpleCNN(num_classes, image_size, in_channels=4)
    else:
        raise ValueError(f"Arquitectura no soportada: {arch_name}")
    return net

def evaluate_on_loader(model, loader, device=device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy().tolist())
            y_pred.extend(preds.cpu().numpy().tolist())

    if len(y_true) != len(y_pred):
        print(f"[WARN] Longitudes distintas en evaluación: y_true={len(y_true)} vs y_pred={len(y_pred)}. Ajustando a longitud mínima.")
        m = min(len(y_true), len(y_pred))
        y_true = y_true[:m]
        y_pred = y_pred[:m]

    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec  = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1   = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    return acc, prec, rec, f1

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=200, patience=10, device=device):
    best_f1 = -1.0
    best_state = None
    patience_counter = 0
    best_epoch = 0

    history = {
        'epoch': [],
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'train_prec': [],
        'val_prec': [],
        'train_rec': [],
        'val_rec': [],
        'train_f1': [],
        'val_f1': []
    }

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        train_preds, train_labels = [], []

        for inputs, labels in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            train_preds.extend(preds.cpu().numpy().tolist())
            train_labels.extend(labels.cpu().numpy().tolist())

        if len(train_labels) != len(train_preds):
            m = min(len(train_labels), len(train_preds))
            train_labels = train_labels[:m]
            train_preds = train_preds[:m]
        train_acc  = accuracy_score(train_labels, train_preds)
        train_prec = precision_score(train_labels, train_preds, average='weighted', zero_division=0)
        train_rec  = recall_score(train_labels, train_preds, average='weighted', zero_division=0)
        train_f1   = f1_score(train_labels, train_preds, average='weighted', zero_division=0)

        model.eval()
        val_running_loss = 0.0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy().tolist())
                val_labels.extend(labels.cpu().numpy().tolist())

        if len(val_labels) != len(val_preds):
            print(f"[WARN] Longitudes distintas en validación: y_true={len(val_labels)} vs y_pred={len(val_preds)}. Ajustando a longitud mínima.")
            m = min(len(val_labels), len(val_preds))
            val_labels = val_labels[:m]
            val_preds  = val_preds[:m]

        val_acc  = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_rec  = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_f1   = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

        history['epoch'].append(epoch)
        history['train_loss'].append(running_loss)
        history['val_loss'].append(val_running_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_prec'].append(train_prec)
        history['val_prec'].append(val_prec)
        history['train_rec'].append(train_rec)
        history['val_rec'].append(val_rec)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch:02d} | train_loss: {running_loss:.4f} | val_loss: {val_running_loss:.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc:.4f} | val_F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
            best_epoch = epoch
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping activado.")
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    model.to(device)

    idx = best_epoch - 1 if best_epoch > 0 else -1
    best_metrics = {
        'train_acc': history['train_acc'][idx],
        'val_acc': history['val_acc'][idx],
        'train_prec': history['train_prec'][idx],
        'val_prec': history['val_prec'][idx],
        'train_rec': history['train_rec'][idx],
        'val_rec': history['val_rec'][idx],
        'train_f1': history['train_f1'][idx],
        'val_f1': history['val_f1'][idx],
        'train_loss': history['train_loss'][idx],
        'val_loss': history['val_loss'][idx]
    }

    return model, best_f1, best_epoch, history, best_metrics

image_sizes   = [224]
batch_sizes   = [16, 32]
learning_rates= [1e-3, 1e-4]
# learning_rates= [5e-3]
epochs        = 200
patience      = 10

architectures = ['resnet50', 'mobilenet', 'simplecnn']
# architectures = ['vit', 'simplecnn']
# architectures = ['resnet50', 'mobilenet', 'vgg16', 'vit', 'simplecnn']
# architectures = ['simplecnn']

all_results = []
all_curves = []
fold_name = os.path.basename(root_fold.rstrip('/\\'))

pretrained_models_cache = {}

for arch in architectures:
    for img_size in image_sizes:
        train_tfms, val_test_tfms = make_transforms(img_size)

        train_ds = ImageFolderRGBA(train_dir, transform=train_tfms)
        val_ds   = ImageFolderRGBA(val_dir,   transform=val_test_tfms)
        test_ds  = ImageFolderRGBA(test_dir,  transform=val_test_tfms)

        for bs in batch_sizes:
            train_loader = DataLoader(train_ds, batch_size=bs, shuffle=True,  num_workers=0, pin_memory=True)
            val_loader   = DataLoader(val_ds,   batch_size=bs, shuffle=False, num_workers=0, pin_memory=True)
            test_loader  = DataLoader(test_ds,  batch_size=bs, shuffle=False, num_workers=0, pin_memory=True)

            for lr in learning_rates:
                print(f"\n=== Entrenando {arch} (pretrained, png4ch) | img={img_size} | bs={bs} | lr={lr} ===")
                start_t = time.time()

                model = build_model_pretrained(arch, num_classes, image_size=img_size, pretrained_models_cache=pretrained_models_cache).to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                model, best_val_f1, best_epoch, history, best_metrics = train_model(
                    model,
                    nn.CrossEntropyLoss(),
                    optimizer,
                    train_loader, val_loader,
                    epochs=epochs, patience=patience, device=device
                )
                train_time = time.time() - start_t

                acc, prec, rec, f1 = evaluate_on_loader(model, test_loader, device=device)

                ts_conf = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

                model_path = f"../models/{foldName}/{arch}_pretrained_png4ch_{ts_conf}_img{img_size}_bs{bs}_lr{lr}.pth"
                torch.save(model.state_dict(), model_path)

                row = {
                    "Fold": fold_name,
                    "Model": arch,
                    "ImageSize": img_size,
                    "BatchSize": bs,
                    "LR": lr,
                    "EpochsUsed": best_epoch,
                    "Patience": patience,
                    "ValF1Best": best_val_f1,
                    "TrainAccuracy": best_metrics['train_acc'],
                    "ValAccuracy": best_metrics['val_acc'],
                    "TrainPrecision": best_metrics['train_prec'],
                    "ValPrecision": best_metrics['val_prec'],
                    "TrainRecall": best_metrics['train_rec'],
                    "ValRecall": best_metrics['val_rec'],
                    "TrainF1": best_metrics['train_f1'],
                    "ValF1": best_metrics['val_f1'],
                    "TrainLoss": best_metrics['train_loss'],
                    "ValLoss": best_metrics['val_loss'],
                    "TestAccuracy": acc,
                    "TestPrecision": prec,
                    "TestRecall": rec,
                    "TestF1": f1,
                    "TrainTimeSec": round(train_time, 2),
                    "Timestamp": ts_conf,
                    "ModelPath": model_path
                }
                all_results.append(row)

                for i in range(len(history['epoch'])):
                    curve_row = {
                        "Fold": fold_name,
                        "Model": arch,
                        "ImageSize": img_size,
                        "BatchSize": bs,
                        "LR": lr,
                        "Epoch": history['epoch'][i],
                        "TrainLoss": history['train_loss'][i],
                        "ValLoss": history['val_loss'][i],
                        "TrainAccuracy": history['train_acc'][i],
                        "ValAccuracy": history['val_acc'][i],
                        "TrainPrecision": history['train_prec'][i],
                        "ValPrecision": history['val_prec'][i],
                        "TrainRecall": history['train_rec'][i],
                        "ValRecall": history['val_rec'][i],
                        "TrainF1": history['train_f1'][i],
                        "ValF1": history['val_f1'][i],
                        "Timestamp": ts_conf,
                        "ModelPath": model_path
                    }
                    all_curves.append(curve_row)

                df = pd.DataFrame(all_results)
                df.to_csv(csv_out_pretrained, index=False)

                df_curves = pd.DataFrame(all_curves)
                df_curves.to_csv(csv_out_curves, index=False)

                print(f"-> Test F1: {f1:.4f} | Val F1 (best): {best_val_f1:.4f} | modelo guardado: {model_path}")

best_by_arch = df.sort_values(by="ValF1Best", ascending=False).groupby("Model").head(1)
print("\nMejores configuraciones por arquitectura (pretrained, png4ch, según ValF1Best):")
print(best_by_arch[["Fold","Model","ImageSize","BatchSize","LR","ValF1Best","TestF1","Timestamp","ModelPath"]])

Usando dispositivo: cuda

=== Entrenando resnet50 (pretrained, png4ch) | img=224 | bs=16 | lr=0.001 ===


KeyboardInterrupt: 

# XAI report

In [ ]:
# ============================================
# XAI para imgShiny
# ============================================

import os
import torch
import numpy as np
import pandas as pd
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import shap
from lime import lime_image
from skimage.segmentation import mark_boundaries


import os
import glob
import time
import datetime
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import vit_b_16

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GradCAM_Pure:
    def __init__(self, model, target_layer, use_pp=False):
        self.model = model
        self.model.eval()
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.use_pp = use_pp
        self.hook_handles = []
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))

    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()

    def __call__(self, input_tensor, class_idx):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if isinstance(output, tuple):
            output = output[0]
        score = output[0, class_idx]
        score.backward(retain_graph=True)
        gradients = self.gradients
        activations = self.activations
        if self.use_pp:
            # GradCAM++ weights
            grads = gradients
            grads_power_2 = grads ** 2
            grads_power_3 = grads ** 3
            sum_activations = torch.sum(activations, dim=(2, 3), keepdim=True)
            eps = 1e-8
            alpha_num = grads_power_2
            alpha_denom = 2 * grads_power_2 + sum_activations * grads_power_3
            alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom) * eps)
            alphas = alpha_num / alpha_denom
            weights = torch.sum(alphas * torch.relu(grads), dim=(2, 3))
        else:
            # GradCAM weights
            weights = torch.mean(gradients, dim=(2, 3))
        cam = torch.zeros(activations.shape[2:], dtype=torch.float32).to(input_tensor.device)
        for i, w in enumerate(weights[0]):
            cam += w * activations[0, i, :, :]
        cam = torch.relu(cam)
        cam -= cam.min()
        cam /= (cam.max() + 1e-8)
        cam = cam.cpu().numpy()
        self.model.zero_grad()
        return cam

# --- Función para superponer el mapa CAM sobre la imagen original ---
def overlay_cam_on_image(img_pil, cam_map, alpha=0.5, colormap=plt.cm.jet):
    cam_map = np.uint8(255 * cam_map)
    cam_map = Image.fromarray(cam_map).resize(img_pil.size, resample=Image.BILINEAR)
    cam_map = np.array(cam_map)
    heatmap = colormap(cam_map / 255.0)[:, :, :3]
    heatmap = np.uint8(255 * heatmap)
    overlay = np.array(img_pil) * (1 - alpha) + heatmap * alpha
    overlay = np.uint8(overlay)
    return overlay

# --- Cargar CSV de mejores modelos ---
csv_name = "../results/metrics_pretrained_imgShiny_20251203_143542_fold1.csv"
best_models = pd.read_csv(csv_name)
best_configs = best_models.sort_values(by='TestF1', ascending=False).groupby('Model').head(1)

timestamp_global = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(f"../results/datasetpaper/{timestamp_global}", exist_ok=True)

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

classes = ['absence', 'presence-defect', 'presence-good']

def load_image(img_path):
    img = Image.open(img_path).convert('RGB')
    tensor = test_transform(img).unsqueeze(0).to(device)
    return img, tensor

# --- Seleccionar 5 imágenes por clase del test set para cada fold ---
for idx, row in best_configs.iterrows():
    model_name = row['Model']
    timestamp = row['Timestamp'] if 'Timestamp' in row else row['ModelPath'].split('_')[1]
    fold = row['Fold'] if 'Fold' in row else 'FoldUnknown'
    model_path = row['ModelPath'] if 'ModelPath' in row else f"../models/{model_name}_{timestamp}.pth"
    test_root = os.path.join("../data/imgShiny/224px", fold, 'test')

    # Cargar modelo
    if model_name == 'mobilenet':
        model = models.mobilenet_v2(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(classes))
        target_layer = model.features[-1]
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=False)
        model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, len(classes))
        target_layer = model.features[-1]
    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, len(classes))
        target_layer = model.layer4[-1]
    else:
        print(f"Modelo no soportado: {model_name}")
        continue

    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    print(f"\nAplicando XAI para {model_name} ({fold})...")

    # Crear carpetas de salida
    for method_name in ['GradCAM', 'GradCAMpp', 'LIME']:
        os.makedirs(f"../results/datasetpaper/{timestamp_global}/{model_name}_{fold}/{method_name}", exist_ok=True)

    # Seleccionar imágenes de test
    sample_images = []
    for cls in classes:
        cls_path = os.path.join(test_root, cls)
        if not os.path.exists(cls_path):
            continue
        imgs = os.listdir(cls_path)[:5]
        sample_images.extend([(os.path.join(cls_path, img), cls) for img in imgs])

    # Aplicar XAI para cada imagen
    for img_path, cls in sample_images:
        img_pil, img_tensor = load_image(img_path)
        target_class = classes.index(cls)

        # Obtener predicción del modelo
        with torch.no_grad():
            output = model(img_tensor)
            if isinstance(output, tuple):
                output = output[0]
            pred_idx = torch.argmax(output, dim=1).item()
            pred_class = classes[pred_idx]

        # Construir nombre de archivo con clase real y predicha
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        name_suffix = f"_real-{cls}_pred-{pred_class}.png"

        # Grad-CAM
        gradcam = GradCAM_Pure(model, target_layer, use_pp=False)
        cam_map = gradcam(img_tensor, target_class)
        overlay = overlay_cam_on_image(img_pil, cam_map, alpha=0.5)
        plt.imshow(overlay)
        plt.axis('off')
        plt.savefig(f"../results/datasetpaper/{timestamp_global}/{model_name}_{fold}/GradCAM/{base_name}{name_suffix}")
        plt.close()
        gradcam.remove_hooks()

        # Grad-CAM++
        gradcampp = GradCAM_Pure(model, target_layer, use_pp=True)
        cam_map_pp = gradcampp(img_tensor, target_class)
        overlay_pp = overlay_cam_on_image(img_pil, cam_map_pp, alpha=0.5)
        plt.imshow(overlay_pp)
        plt.axis('off')
        plt.savefig(f"../results/datasetpaper/{timestamp_global}/{model_name}_{fold}/GradCAMpp/{base_name}{name_suffix}")
        plt.close()
        gradcampp.remove_hooks()

        # LIME
        explainer = lime_image.LimeImageExplainer()
        img_np = np.array(img_pil)

        def predict_fn(images):
            batch = torch.stack([test_transform(Image.fromarray(img)).to(device) for img in images])
            with torch.no_grad():
                outputs = model(batch)
            return outputs.cpu().numpy()

        explanation = explainer.explain_instance(
            img_np,
            predict_fn,
            top_labels=1,
            hide_color=0,
            num_samples=1000
        )

        # Obtener imagen y máscara
        temp, mask = explanation.get_image_and_mask(
            explanation.top_labels[0],
            positive_only=True,
            num_features=5,
            hide_rest=False
        )

        # Superponer bordes de superpixeles
        from scipy.ndimage import binary_dilation

        # Aumentar el grosor de los bordes (ajusta iterations para más grosor)
        dilated_mask = binary_dilation(mask, iterations=3)

        # Aplicar transparencia del 30% a la imagen temp (convertir a float, escalar, y volver a uint8)
        temp_alpha = temp.astype(float) * 0.7 + 255 * 0.3  # 70% imagen, 30% blanco para remarcar LIME
        temp_alpha = np.clip(temp_alpha, 0, 255).astype(np.uint8)

        # Mostrar con bordes gruesos y fondo atenuado
        plt.imshow(mark_boundaries(temp_alpha, dilated_mask, color=(1, 0, 0), mode='thick'))
        plt.axis('off')

        # Guardar resultado
        plt.savefig(f"../results/datasetpaper/{timestamp_global}/{model_name}_{fold}/LIME/{base_name}{name_suffix}")
        plt.close()

        # # SHAP
        # background = torch.cat([img_tensor for _ in range(5)], dim=0)
        # e = shap.DeepExplainer(model, background)
        # shap_values = e.shap_values(img_tensor)
        # shap.image_plot(shap_values, np.array([np.array(img_pil)]), show=False)
        # plt.savefig(f"../results/XAI/{model_name}_{fold}/SHAP/{base_name}{name_suffix}")
        # plt.close()

print("Visualizaciones XAI (sin torchcam) generadas y guardadas en '../results/XAI/'.")

c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torchvisio


Aplicando XAI (sin torchcam) para resnet50 (orFold1)...


c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 1000/1000 [00:05<00:00, 184.99it/s]

c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
c:\Users\nalex\OneDrive\Documentos\PhD\VSpro


Aplicando XAI (sin torchcam) para mobilenet (orFold1)...


c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 1000/1000 [00:05<00:00, 167.28it/s]
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)

c:\Users\nalex\OneDrive\Documentos\PhD\VSpro

Modelo no soportado: simplecnn

Aplicando XAI (sin torchcam) para vgg16 (orFold1)...

Aplicando XAI (sin torchcam) para vgg16 (orFold1)...


100%|██████████| 1000/1000 [00:06<00:00, 145.35it/s]

100%|██████████| 1000/1000 [00:08<00:00, 122.62it/s]

100%|██████████| 1000/1000 [00:08<00:00, 119.81it/s]

100%|██████████| 1000/1000 [00:07<00:00, 138.73it/s]

100%|██████████| 1000/1000 [00:07<00:00, 136.42it/s]

100%|██████████| 1000/1000 [00:07<00:00, 131.32it/s]

100%|██████████| 1000/1000 [00:07<00:00, 127.84it/s]

100%|██████████| 1000/1000 [00:08<00:00, 121.86it/s]

100%|██████████| 1000/1000 [00:07<00:00, 137.91it/s]

100%|██████████| 1000/1000 [00:05<00:00, 172.72it/s]

100%|██████████| 1000/1000 [00:05<00:00, 171.59it/s]

100%|██████████| 1000/1000 [00:05<00:00, 173.76it/s]

100%|██████████| 1000/1000 [00:05<00:00, 197.83it/s]

100%|██████████| 1000/1000 [00:05<00:00, 174.82it/s]



Visualizaciones XAI (sin torchcam) generadas y guardadas en '../results/XAI/'.


In [ ]:
# ============================================
# XAI Report for RGBA images (png4ch, 4 canales)
# ============================================
import os
import torch
import numpy as np
import pandas as pd
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import shap
from lime import lime_image
from skimage.segmentation import mark_boundaries
from scipy.ndimage import binary_dilation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GradCAM_Pure:
    def __init__(self, model, target_layer, use_pp=False):
        self.model = model
        self.model.eval()
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.use_pp = use_pp
        self.hook_handles = []
        self._register_hooks()
    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()
        self.hook_handles.append(self.target_layer.register_forward_hook(forward_hook))
        self.hook_handles.append(self.target_layer.register_backward_hook(backward_hook))
    def remove_hooks(self):
        for handle in self.hook_handles:
            handle.remove()
    def __call__(self, input_tensor, class_idx):
        self.model.zero_grad()
        output = self.model(input_tensor)
        if isinstance(output, tuple):
            output = output[0]
        score = output[0, class_idx]
        score.backward(retain_graph=True)
        gradients = self.gradients
        activations = self.activations
        if self.use_pp:
            grads = gradients
            grads_power_2 = grads ** 2
            grads_power_3 = grads ** 3
            sum_activations = torch.sum(activations, dim=(2, 3), keepdim=True)
            eps = 1e-8
            alpha_num = grads_power_2
            alpha_denom = 2 * grads_power_2 + sum_activations * grads_power_3
            alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom) * eps)
            alphas = alpha_num / alpha_denom
            weights = torch.sum(alphas * torch.relu(grads), dim=(2, 3))
        else:
            weights = torch.mean(gradients, dim=(2, 3))
        cam = torch.zeros(activations.shape[2:], dtype=torch.float32).to(input_tensor.device)
        for i, w in enumerate(weights[0]):
            cam += w * activations[0, i, :, :]
        cam = torch.relu(cam)
        cam -= cam.min()
        cam /= (cam.max() + 1e-8)
        cam = cam.cpu().numpy()
        self.model.zero_grad()
        return cam

def overlay_cam_on_image(img_pil, cam_map, alpha=0.5, colormap=plt.cm.jet):
    cam_map = np.uint8(255 * cam_map)
    cam_map = Image.fromarray(cam_map).resize(img_pil.size, resample=Image.BILINEAR)
    cam_map = np.array(cam_map)
    heatmap = colormap(cam_map / 255.0)[:, :, :3]
    heatmap = np.uint8(255 * heatmap)
    overlay = np.array(img_pil)[..., :3] * (1 - alpha) + heatmap * alpha
    overlay = np.uint8(overlay)
    return overlay

def pil_loader_rgba(path):
    img = Image.open(path)
    arr = np.array(img)
    if arr.ndim == 2:
        arr = np.stack([arr]*4, axis=-1)
    if arr.shape[2] == 3:
        arr = np.concatenate([arr, np.zeros_like(arr[..., :1])], axis=-1)
    elif arr.shape[2] == 1:
        arr = np.concatenate([arr]*4, axis=-1)
    elif arr.shape[2] != 4:
        raise ValueError(f"Imagen con {arr.shape[2]} canales no soportada")
    arr = arr.astype(np.float32) / 255.0
    tensor = torch.from_numpy(arr).permute(2,0,1)
    return tensor

csv_name = "../results/metrics_pretrained_png4ch_20251203_233013_fold1.csv"
import glob
csv_files = glob.glob(csv_name)
if not csv_files:
    raise FileNotFoundError("No se encontró el archivo de métricas para png4ch")
csv_name = csv_files[-1]
best_models = pd.read_csv(csv_name)
best_configs = best_models.sort_values(by='TestF1', ascending=False).groupby('Model').head(1)

timestamp_global = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(f"../results/dataset_png4ch/{timestamp_global}", exist_ok=True)

classes = ['absence', 'presence-defect', 'presence-good']

def load_image_rgba(img_path):
    tensor = pil_loader_rgba(img_path)
    img = Image.open(img_path).convert('RGBA')
    return img, tensor.unsqueeze(0).to(device)

for idx, row in best_configs.iterrows():
    model_name = row['Model']
    timestamp = row['Timestamp'] if 'Timestamp' in row else row['ModelPath'].split('_')[1]
    fold = row['Fold'] if 'Fold' in row else 'FoldUnknown'
    model_path = row['ModelPath'] if 'ModelPath' in row else f"../models/{model_name}_{timestamp}.pth"
    test_root = os.path.join("../data/png4ch/224px", fold, 'test')

    from torchvision.models import mobilenet_v2, vgg16, resnet50, vit_b_16
    from torch import nn
    if model_name == 'mobilenet':
        proj = nn.Conv2d(4, 3, kernel_size=1, bias=True)
        backbone = mobilenet_v2(pretrained=False)
        backbone.classifier[1] = nn.Linear(backbone.classifier[1].in_features, len(classes))
        target_layer = backbone.features[-1]
        model = nn.Sequential(proj, backbone)
    elif model_name == 'vgg16':
        proj = nn.Conv2d(4, 3, kernel_size=1, bias=True)
        backbone = vgg16(pretrained=False)
        backbone.classifier[6] = nn.Linear(backbone.classifier[6].in_features, len(classes))
        target_layer = backbone.features[-1]
        model = nn.Sequential(proj, backbone)
    elif model_name == 'resnet50':
        proj = nn.Conv2d(4, 3, kernel_size=1, bias=True)
        backbone = resnet50(pretrained=False)
        backbone.fc = nn.Linear(backbone.fc.in_features, len(classes))
        target_layer = backbone.layer4[-1]
        model = nn.Sequential(proj, backbone)
    else:
        print(f"Modelo no soportado: {model_name}")
        continue

    # --- Cargar state_dict y mapear claves ---
    state = torch.load(model_path, map_location=device)
    model_state = model.state_dict()
    if set(state.keys()) != set(model_state.keys()):
        new_state = {}
        for k in state.keys():
            if k.startswith('0.proj.'):
                newk = k.replace('0.proj.', '0.')
                new_state[newk] = state[k]
            else:
                new_state[k] = state[k]
        state = new_state
    model.load_state_dict(state, strict=False)
    model = model.to(device)
    model.eval()

    print(f"\nAplicando XAI (sin torchcam) para {model_name} ({fold}, RGBA)...")

    for method_name in ['GradCAM', 'GradCAMpp', 'LIME']:
        os.makedirs(f"../results/dataset_png4ch/{timestamp_global}/{model_name}_{fold}/{method_name}", exist_ok=True)

    sample_images = []
    for cls in classes:
        cls_path = os.path.join(test_root, cls)
        if not os.path.exists(cls_path):
            continue
        imgs = os.listdir(cls_path)[:5]
        sample_images.extend([(os.path.join(cls_path, img), cls) for img in imgs])

    for img_path, cls in sample_images:
        img_pil, img_tensor = load_image_rgba(img_path)
        target_class = classes.index(cls)

        with torch.no_grad():
            output = model(img_tensor)
            if isinstance(output, tuple):
                output = output[0]
            pred_idx = torch.argmax(output, dim=1).item()
            pred_class = classes[pred_idx]

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        name_suffix = f"_real-{cls}_pred-{pred_class}.png"

        gradcam = GradCAM_Pure(model, target_layer, use_pp=False)
        cam_map = gradcam(img_tensor, target_class)
        overlay = overlay_cam_on_image(img_pil, cam_map, alpha=0.5)
        plt.imshow(overlay)
        plt.axis('off')
        plt.savefig(f"../results/dataset_png4ch/{timestamp_global}/{model_name}_{fold}/GradCAM/{base_name}{name_suffix}")
        plt.close()
        gradcam.remove_hooks()

        gradcampp = GradCAM_Pure(model, target_layer, use_pp=True)
        cam_map_pp = gradcampp(img_tensor, target_class)
        overlay_pp = overlay_cam_on_image(img_pil, cam_map_pp, alpha=0.5)
        plt.imshow(overlay_pp)
        plt.axis('off')
        plt.savefig(f"../results/dataset_png4ch/{timestamp_global}/{model_name}_{fold}/GradCAMpp/{base_name}{name_suffix}")
        plt.close()
        gradcampp.remove_hooks()

        explainer = lime_image.LimeImageExplainer()
        img_np = np.array(img_pil)[..., :3]
        def predict_fn(images):
            batch = torch.stack([torch.from_numpy(img.transpose(2,0,1)).float().div(255).unsqueeze(0)[0] for img in images]).to(device)
            if batch.shape[1] == 3:
                batch = torch.cat([batch, torch.zeros(batch.shape[0],1,batch.shape[2],batch.shape[3], device=batch.device)], dim=1)
            with torch.no_grad():
                outputs = model(batch)
            return outputs.cpu().numpy()

        explanation = explainer.explain_instance(
            img_np,
            predict_fn,
            top_labels=1,
            hide_color=0,
            num_samples=1000
        )

        temp, mask = explanation.get_image_and_mask(
            explanation.top_labels[0],
            positive_only=True,
            num_features=5,
            hide_rest=False
        )

        dilated_mask = binary_dilation(mask, iterations=3)
        temp_alpha = temp.astype(float) * 0.7 + 255 * 0.3
        temp_alpha = np.clip(temp_alpha, 0, 255).astype(np.uint8)

        plt.imshow(mark_boundaries(temp_alpha, dilated_mask, color=(1, 0, 0), mode='thick'))
        plt.axis('off')
        plt.savefig(f"../results/dataset_png4ch/{timestamp_global}/{model_name}_{fold}/LIME/{base_name}{name_suffix}")
        plt.close()

print("Visualizaciones XAI (RGBA, sin torchcam) generadas y guardadas en '../results/XAI_png4ch/'.")

c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Aplicando XAI (sin torchcam) para mobilenet (orFold1, RGBA)...


c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 1000/1000 [00:05<00:00, 169.04it/s]

c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
c:\Users\nalex\OneDrive\Documentos\PhD\VSpro


Aplicando XAI (sin torchcam) para resnet50 (orFold1, RGBA)...


c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
100%|██████████| 1000/1000 [00:08<00:00, 121.31it/s]
c:\Users\nalex\OneDrive\Documentos\PhD\VSprojects\BGA_defectClass\venv\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)

c:\Users\nalex\OneDrive\Documentos\PhD\VSpro

Modelo no soportado: simplecnn
Visualizaciones XAI (RGBA, sin torchcam) generadas y guardadas en '../results/XAI_png4ch/'.


In [10]:
# --- Lectura de una imagen aleatoria por subcarpeta para identificar número de canales ---
import os
import random
from PIL import Image

# base_dir = "../data/png4ch/original_copia"
# base_dir = "../data/png4ch/original"
base_dir = "../data/png4ch/224px/orFold1/train_aug"

for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
    images = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff'))]
    if not images:
        print(f"No hay imágenes en {subfolder_path}")
        continue
    img_file = random.choice(images)
    img_path = os.path.join(subfolder_path, img_file)
    try:
        img = Image.open(img_path)
        print(f"{subfolder}: {img_file} -> {img.mode} ({img.getbands()}) -> shape: {img.size}, canales: {len(img.getbands())}")
    except Exception as e:
        print(f"Error abriendo {img_path}: {e}")

absence: aug_0096.png -> RGBA (('R', 'G', 'B', 'A')) -> shape: (224, 224), canales: 4
presence-defect: aug_0224.png -> RGBA (('R', 'G', 'B', 'A')) -> shape: (224, 224), canales: 4
presence-good: aug_0239.png -> RGBA (('R', 'G', 'B', 'A')) -> shape: (224, 224), canales: 4
